# 前提条件の確認

技術的な前提条件を確認する。

## ネットワークの確認

ホストマシンにNICが2つあって、1つは上流に接続、1つはブリッジに繋がった状態であることを確認する。

/etc/network/interfacesの例
```
auto lo
iface lo inet loopback

# 1つは上流に接続
auto eth0
iface eth0 inet dhcp

# 1つはブリッジに繋がった状態
auto eth1
iface eth1 inet manual

auto br14
iface br14 inet static
address 192.168.14.1
netmask 255.255.255.0
bridge_ports eth1
bridge_maxwait 0
bridge_fd 0
bridge_stp off
```

jupiterは2つのネットワークを全て繋ぐ設定で起動しているので、192.168.14.0/24のネットワークに繋がったインタフェースが見えていることと、上流への疎通があってインターネットに出れることを確認しておく。

In [ ]:
!ip a
!test $(ip a | grep 192.168.14 | wc -l) -eq 1

!ping -c 4 yahoo.jp
!test $(ping -c 4 yahoo.jp | grep "4 packets received" | wc -l) -eq 1

## コンテナの起動確認

同梱のdocker-compose.ymlでコンテナ一式が起動した状態であることを確認する。

[同梱のdocker-compose.yml](/edit/wifi/docker-compose.yml)

docker-composeで作成された各種リソースには、起動した場所（docker-compose.ymlの場所）を示す文字列が接頭語として追加される。このnotebookでは/rootで起動して、接頭語がrootになっている前提とする。

コンテナ起動状態を確認するため、dockerコマンドをインストール（jupyterには/var/run/docker.sockをマウントして、dockerコマンドを使えるようにしてある）し、juyter、vyos、muninのコンテナが動いていることと、dockerの内部ネットワーク、root_default、root_wifiが存在することを確認しておく。

In [ ]:
!curl -fsSL https://download.docker.com/linux/static/stable/x86_64/docker-17.09.0-ce.tgz \
    | sudo tar -xzC /usr/local/bin --strip=1 docker/docker

!sudo docker ps
!test $(sudo docker ps | grep jupyter | wc -l) -eq 1
!test $(sudo docker ps | grep vyos | wc -l) -eq 1
!test $(sudo docker ps | grep munin | wc -l) -eq 1

!sudo docker network ls
!test $(sudo docker network ls | grep root_default | wc -l) -eq 1
!test $(sudo docker network ls | grep root_wifi | wc -l) -eq 1